In [ ]:
# data access

import xarray as xr

In [ ]:
# visualisation

from cartopy import crs
import display
import matplotlib.pyplot as plt

In [ ]:
PROJECTION = crs.Miller()
area = {"latitude": slice(-5, 5), "longitude": slice(-120, -80)}
el_nino_event_range = ("1997-04", "1998-04")

In [ ]:
# data access configuration

dataset = "s3://cmems-reanalysis/cmems_mod_glo_phy_my_0.083_P1D-m.zarr"

In [ ]:
ds = xr.open_dataset(dataset, chunks={}, engine="zarr").astype("float32")
ds

In [ ]:
%%time

zos_pacific_area = ds.zos.sel(**area)
zos_mean = zos_pacific_area.mean(dim="time")
zos_anomaly = zos_pacific_area - zos_mean
zos_anomaly_el_nino_event = zos_anomaly.sel(time=slice(el_nino_event_range[0], el_nino_event_range[1])).mean(dim="time").compute()

In [ ]:
# display.map(zos_anomaly_el_nino_event, vmax=None, title="1997-1998 El Nino-event")
_, ax = plt.subplots(
    figsize=(10, 8),
    subplot_kw={"projection": PROJECTION},
)
zos_anomaly_el_nino_event.plot(
    ax=ax, 
    cmap="Blues", 
    transform=crs.PlateCarree(), 
    cbar_kwargs={"orientation": "horizontal", "pad": 0.05, "aspect": 40, "label": "Sea Surface Height anomaly [m]"},
)
ax.coastlines()
ax.set_title("El Nino event (1997-1998)")
plt.show()

In [ ]:
%%time

el_nino_area_ds = ds.sel(**area)
zos_monthly = el_nino_area_ds.zos.resample(time="1M").mean()
zos_monthly_spatial_mean = zos_monthly.mean(dim=["latitude", "longitude"])
zos_mean = zos_monthly_spatial_mean.mean()
zos_anomaly = (zos_monthly_spatial_mean - zos_mean).compute()

In [ ]:
plt.plot(zos_anomaly['time'], zos_anomaly)
plt.xlabel('Year')
plt.ylabel('Sea Surface Height anomaly [m]')
plt.xticks(rotation=45)

plt.show()